### Python setup

See `README.md` for python environment setup instructions on OSX using `pyenv` in VSCode.  If you've got another way of getting a modern, viable python environment, feel free to use it.

Here are the versions that I'm using when running this notebook:

In [1]:
import platform
import psutil

print(platform.python_version())

memory_info = psutil.virtual_memory()
print(f"Total memory: {memory_info.total / (1024 ** 3):.2f} GB")
print(f"Available memory: {memory_info.available / (1024 ** 3):.2f} GB")
print(f"Used memory: {memory_info.used / (1024 ** 3):.2f} GB")
print(f"Memory percent: {memory_info.percent}%")

3.12.2
Total memory: 64.00 GB
Available memory: 34.60 GB
Used memory: 27.83 GB
Memory percent: 45.9%


### Generate a self-signed cert that we can use for the https redirect during the OAuth login

You'll need to tell your browser to "trust" this cert after the redirect

In [2]:
import os
import subprocess
import shutil


def generate_key():
    if shutil.which("openssl") is None:
        print(
            "openssl is not found. Please install openssl and make sure it's on your system path. See: https://wiki.openssl.org/index.php/Binaries"
        )
        return

    if os.path.isfile("key.pem"):
        print(
            "key.pem already exists, not regenerating it. Delete it if you'd like to regenerate it."
        )
    else:
        command = 'openssl req -x509 -newkey rsa:4096 -keyout key.pem -out cert.pem -days 365 -nodes -subj "/C=US/ST=California/L=San Francisco/O=My Company/OU=My Division/CN=localhost"'
        process = subprocess.Popen(command, shell=True)
        process.wait()


generate_key()

key.pem already exists, not regenerating it. Delete it if you'd like to regenerate it.


### API Key and Client ID

You'll need an API Key and Client ID to interact with Bungie's API.  You can generate one on Bungie's website: https://www.bungie.net/en/Application

You can generate an application with the "Public" OAuth type with a redirect url of: `https://localhost:7777/oauth-redirect`  - this will be used after the oauth login to get the oauth token that will be used in API requests

The only scope necessary is read access: `Read your Destiny 2 information (Vault, Inventory, and Vendors), as well as Destiny 1 Vault and Inventory data` 

This is what it should look like:

![Image](images/oauth-app-settings.png)


Once you create your application, you should get an `OAuth client_id` and an `API Key`, values for these should be put into `config.json` along with your Bungie `username` (Steam usernames will have a 4-digit hash like `#1234` on the end).  It should look like this:

```
{ "client_id": "your_client_id", "api_key": "your_api_key", "username": "your_username#1234"}
```

This same username is used on things like https://dungeon.report to find your user.

This next cell will read that file in and make the `client_id`, `api_key`, and `username` values available to the rest of the calls.


In [3]:
from src import config

client_id, api_key, username = config.load_config()

print(f"client_id length: {len(client_id)}")
print(f"api_key length: {len(api_key)}")
print(f"username: {username}")

client_id length: 5
api_key length: 32
username: ted#2434


### Login with OAuth to get a token that'll last for 1 hour

In [4]:
from src.bungie_oauth import BungieAuth
import datetime

# perform oauth login to get the access token used in later requests.  It is good for 1 hour
print(
    "We're using a self-signed certificate to run an HTTPS server on localhost, you'll need to accept the certificate in your browser."
)
access_token = BungieAuth(client_id).refresh_oauth_token()

# token is good for 1 hour, print out the time that it expires
expiration_time = datetime.datetime.now() + datetime.timedelta(hours=1)
print(
    f"Access token successfully acquired at: {datetime.datetime.now().isoformat()} and expires at: {expiration_time.isoformat()}"
)

We're using a self-signed certificate to run an HTTPS server on localhost, you'll need to accept the certificate in your browser.
Please go to the following URL and authorize the app: https://www.bungie.net/en/oauth/authorize?client_id=22927&response_type=code&state=TS_1B-yBdKNZDx-5l2uPle5Q4Q6GXlovTt6o4jMnvFy5sb1TaYaqli8zegCCqASqTeKoy1Tk3xC68E0RYuDwgTIfSBaygXxb6g9nkbYyb2YvSkmUn24n9q0fzGYTJFs6Os0QcyvmUCxBXxpGZfnsPrsSL5i_ChMw9mGTsG1u2oU=&redirect_uri=https://localhost:7777/
Stopping HTTPS server
Access token successfully acquired at: 2024-06-06T12:19:45.213207 and expires at: 2024-06-06T13:19:45.212997


### Now we're ready to talk to Bungie's API

In [5]:
from src.bungie_api import BungieApi

api = BungieApi(api_key, access_token)

membership_id, profile_type = api.get_primary_membership_id_and_type(username)

if membership_id is not None and profile_type is not None:
    print(f"Membership ID: {membership_id}, Profile Type: {profile_type}")
    api.get_character_ids_and_classes(membership_id, profile_type)

Checking membership ID 4611686018436088066 with membership type 2
Crosave override found for 4611686018436088066
Membership ID: 4611686018436088066, Profile Type: 2


In [6]:
# download the character profile for this membership_id
import json

# access_token, profile_type, and membership_id should be retrieved above
# using the login_and_get_token and get_primary_membership_id_and_type functions

# https://bungie-net.github.io/multi/schema_Destiny-DestinyComponentType.html#schema_Destiny-DestinyComponentType
# 100 = profile.data.userInfo
# 102 = profileInventory.data.items
# 201 = characterInventories.data[character_id].items
# 205 = characterEquipment.data[character_id].items
# 300 = itemComponents.instances
# 305 = profilePlugSets.data.plugs, characterPlugSets.data[character_id].plugs, itemComponents.sockets

# this gives us all of the information we need for vault armor for this user
profile = api.get_profile(
    access_token, profile_type, membership_id, [100, 102, 201, 205, 300, 305, 309]
)

# retrieve the manifest and item/stat definitions that will be joined with profile data to determine what armor you have in your vault
item_definitions, stat_definitions = api.get_static_definitions()

os.makedirs("data", exist_ok=True)

# dump the profile out as json into the data directory
with open("data/profile.json", "w") as file:
    json.dump(profile, file, indent=4)

# dump the manifest definitions out as json into the data directory
with open("data/item_definitions.json", "w") as file:
    json.dump(item_definitions, file, indent=4)

# dump the stat definitions out as json into the data directory
with open("data/stat_definitions.json", "w") as file:
    json.dump(stat_definitions, file, indent=4)

print("Character profile loaded at:", profile["responseMintedTimestamp"])

Character profile loaded at: 2024-06-06T17:19:50.728Z
